In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
from bokeh.models import HoverTool

hv.extension("bokeh")

In [ ]:
n_channels, n_seconds, fs = 25, 30, 512

data = np.random.random((n_channels, 15360))
time = np.arange(n_seconds * fs) / fs
channels = [f"EEG {n+1:02d}" for n in range(n_channels)]
offset = np.std(data) * 6

hover = HoverTool(
    tooltips=[("Channel", "@channel"), ("Time", "$x s"), ("Amplitude", "@original_amplitude µV")]
)
shared_opts = dict(color="black", line_width=1, tools=[hover, "xwheel_zoom"], shared_axes=False)

In [ ]:
# Overlay

channel_curves = []
for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    ds = hv.Dataset(
        (time, offset_data, channel_data, channels[i]),
        ["Time", "Amplitude", "original_amplitude", "channel"],
    )
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(**shared_opts)
    )

plot1 = hv.Overlay(channel_curves, kdims="Channel")
type(plot1)

In [ ]:
# NdOverlay
data2 = data + (np.arange(len(data))[:, np.newaxis] * offset)

xds = xr.Dataset(
    coords={"channel": channels, "Time": time},
    data_vars={
        "Amplitude": (("channel", "Time"), data2),
        # "original_amplitude": (("channel", "Time"), data),
    },
)

plot2 = (
    hv.Dataset(xds)
    .to(hv.Curve, groupby="channel")
    .overlay()
    .opts(hv.opts.Curve(**shared_opts))
).opts(tools=[hover], show_legend=False)
type(plot2)

In [ ]:
(plot1 + plot2).opts(shared_axes=False)